In [4]:
import os
import pandas as pd
import numpy as np
import json
import pickle 
import torch
import itertools
import time

In [5]:
PATH_data = "/data/scratch/skatz/PROJECTS/methylnet/1_healthyVAE/data/GSE87571/train_val_test_sets/"

In [15]:
CHR = "chr14"
PATH_results = f"logs/optimisation/{CHR}/coarse"
os.makedirs(PATH_results, exist_ok=True)
param_grid = dict()

### Step 1: how many CpGs in chromosome of choice?
with open(os.path.join(PATH_data, f"{CHR}_train_methyl_array.pkl"), "rb") as f: train_dataset = pickle.load(f) #
num_cpgs = train_dataset["beta"].shape[1]
print(f"Number of CpGs for chromosome_{CHR}: \t{num_cpgs}")

### Step 2: design hidden layers (fixed to two hidden layers)
reduction_rate_hidden_layers = [0.30, 0.20]
hidden_1 = int(round(num_cpgs*reduction_rate_hidden_layers[0], -1))  # Hidden layer 1: reduction to 30% 
hidden_2 = int(round(hidden_1*reduction_rate_hidden_layers[1], -1))  # Hidden layer 2: reduction to another 20% --> total reduction to 6%
param_grid["hidden_layer_encoder_topology"] = [hidden_1, hidden_2]
print(f"Hidden layers: \t{hidden_1} - {hidden_2}")

### Step 3: design latSizes to try for coarse grained search
reduction_latSize_coarse = [2,4,8,16,32,64]
latSize_coarse = [int(round(hidden_2/ele, -1)) for ele in reduction_latSize_coarse]
if latSize_coarse[-1] == 0: ### don't allow it to be 0
    latSize_coarse[-1] = 5
param_grid["latentSize_coarse"] = latSize_coarse
print(f"LatSizes (coarse search): \t {latSize_coarse}")

### Step 4: decide on lr and dropout rates to try
### lr_scan = [1e-4,1e-3,1e-2,1e-1]  ##### **NOTE from later experiments:** does **not** train with too high learning rates (0.1, 0.01)
lr_scan = [1e-4, 5e-4, 1e-3, 5e-3] 
param_grid["lr"] = lr_scan
dropout_scan = [0.1,0.3,0.5]
param_grid["dropout"] = dropout_scan

### Save parameter grid in file for later documentation
with open(f"{PATH_results}/param_grid_coarse.json", "w") as f: f.write(json.dumps(param_grid, indent="\t"))


Number of CpGs for chromosome_chr14: 	9417
Hidden layers: 	2830 - 570
LatSizes (coarse search): 	 [280, 140, 70, 40, 20, 10]


In [16]:
### Step 5: generate combinations of parameters and replace in file; standaline script?
### Load parameter grid
with open(f"{PATH_results}/param_grid_coarse.json", "r") as f: param_grid_comb=json.loads(f.read())

### Generate combinations; remove 'hidden_layer_encoder_topology' before, as it should be fixed
param_grid = param_grid_comb.copy()
param_grid_comb.pop("hidden_layer_encoder_topology", None)
combs_coarse = [dict(zip(param_grid_comb.keys(), values)) for values in itertools.product(*param_grid_comb.values())]
print(f"Number of combinations: {len(combs_coarse)}")

### Generate submit.sh with combinations
os.makedirs(f"{PATH_results}/submit", exist_ok=True)
with open("submit_template.sh", "r") as f: template=f.read()
for i in range(len(combs_coarse)):                          
    latSize = str(combs_coarse[i]["latentSize_coarse"])
    lr = str(combs_coarse[i]["lr"])
    dropr = str(combs_coarse[i]["dropout"])
    
    ### Generate run name - combination of parameter settings
    fileName = f"latSize_{latSize}_lr_{lr}_dropr_{dropr}"
    ### Replace in template file
    template_updated = template.replace("$PATH", str(PATH_results+"/"+fileName)) \
                               .replace("$CHR",  str(CHR)) \
                               .replace("$HIDDEN_1", str(param_grid["hidden_layer_encoder_topology"][0])) \
                               .replace("$HIDDEN_2", str(param_grid["hidden_layer_encoder_topology"][1])) \
                               .replace("$LATSIZE", latSize) \
                               .replace("$LR", lr) \
                               .replace("$DROPR", dropr)
    with open(f"{PATH_results}/submit/{fileName}.sh", "w") as f: f.write(template_updated)
    print(f"Wrote file \t{fileName}")

Number of combinations: 72
Wrote file 	latSize_280_lr_0.0001_dropr_0.1
Wrote file 	latSize_280_lr_0.0001_dropr_0.3
Wrote file 	latSize_280_lr_0.0001_dropr_0.5
Wrote file 	latSize_280_lr_0.0005_dropr_0.1
Wrote file 	latSize_280_lr_0.0005_dropr_0.3
Wrote file 	latSize_280_lr_0.0005_dropr_0.5
Wrote file 	latSize_280_lr_0.001_dropr_0.1
Wrote file 	latSize_280_lr_0.001_dropr_0.3
Wrote file 	latSize_280_lr_0.001_dropr_0.5
Wrote file 	latSize_280_lr_0.005_dropr_0.1
Wrote file 	latSize_280_lr_0.005_dropr_0.3
Wrote file 	latSize_280_lr_0.005_dropr_0.5
Wrote file 	latSize_140_lr_0.0001_dropr_0.1
Wrote file 	latSize_140_lr_0.0001_dropr_0.3
Wrote file 	latSize_140_lr_0.0001_dropr_0.5
Wrote file 	latSize_140_lr_0.0005_dropr_0.1
Wrote file 	latSize_140_lr_0.0005_dropr_0.3
Wrote file 	latSize_140_lr_0.0005_dropr_0.5
Wrote file 	latSize_140_lr_0.001_dropr_0.1
Wrote file 	latSize_140_lr_0.001_dropr_0.3
Wrote file 	latSize_140_lr_0.001_dropr_0.5
Wrote file 	latSize_140_lr_0.005_dropr_0.1
Wrote file 	lat

### DEV: script submitting and tracking progress

In [10]:
### Initialise log file only once!!
#with open(f"{path_log}/log_submitted_jobs.txt", "w") as f: f.write("")

In [14]:
import subprocess
joblimit = 3

### Set path 
path_log = f"{PATH_results}/submit"

### Check number of jobs submitted to the cluster
numRunningJobs = int(subprocess.run("squeue | grep 'katz' | wc -l", shell=True, capture_output=True, text=True).stdout.strip("\n"))

### Loop over jobs and chekc if they have been subitted; if not: submit and add to `log_submitted_jobs.txt`
for file in os.listdir(path_log):
    if file.endswith(".sh"):
        ### if less than 10 jobs - submit new ones!
        while numRunningJobs < joblimit:  
            #print(file)
            ### Get jobs that have already been submitted
            with open(f"{path_log}/log_submitted_jobs.txt", "r") as f: jobsFinished = f.read().splitlines()
            if file not in jobsFinished:
                ### Submit and record in log
                call = f"sbatch {path_log}/{str(file)}"
                subprocess.run(call, shell=True) 
                with open(f"{path_log}/log_submitted_jobs.txt", "w") as f: [f.write(line+"\n") for line in jobsFinished + [file]]
            else: break
            ## recheck submitted jobs to break loop
            time.sleep(1)
            numRunningJobs = int(subprocess.run("squeue | grep 'katz' | wc -l", shell=True, capture_output=True, text=True).stdout.strip("\n"))
else: 
    print(f"Cluster full \t\t current jobs: \t {numRunningJobs}\t-\t limit for myself: {joblimit}")
    
print(f"\nNumber of finished jobs: \t{len(jobsFinished)} / 72")

Submitted batch job 148069
Cluster full 		 current jobs: 	 3	-	 limit for myself: 3

Number of finished jobs: 	2 / 72


In [143]:
with open("submit_template.sh", "r") as f: template=f.read()
for i in range(3):
    latSize = str(combs_coarse[i]["latentSize_coarse"])
    lr = str(combs_coarse[i]["lr"])
    dropr = str(combs_coarse[i]["dropout"])
    
    ### Generate run name - combination of parameter settings
    fileName = f"latSize_{latSize}_lr_{lr}_dropr_{dropr}"
    ### Replace in template file
    template_updated = template.replace("$PATH", str(PATH_results+"/"+fileName)) \
                               .replace("$CHR",  str(CHR)) \
                               .replace("$HIDDEN_1", str(param_grid["hidden_layer_encoder_topology"][0])) \
                               .replace("$HIDDEN_2", str(param_grid["hidden_layer_encoder_topology"][1])) \
                               .replace("$LATSIZE", latSize) \
                               .replace("$LR", lr) \
                               .replace("$DROPR", dropr)
    with open(f"{PATH_results}/submit/submit_{fileName}.sh", "w") as f: f.write(template_updated)

{'latentSize_coarse': 200, 'lr': 0.0001, 'dropout': 0.1}
{'latentSize_coarse': 200, 'lr': 0.0001, 'dropout': 0.3}
{'latentSize_coarse': 200, 'lr': 0.0001, 'dropout': 0.5}


In [139]:
PATH_results+"/"+fileName

'logs/optimisation/chr20/latSize_200_lr_0.0001_dropr_0.5'

In [135]:
latSize_{combs_coarse[i]["latentSize_coarse"]}_

200

In [109]:
## Experiment: try reading file and removing things
with open("submit_template.sh", "r") as f: template=f.read()
template_updated = template.replace("$CHR", str(CHR)) \
                           .replace("$HIDDEN_1", str(15))
with open(f"submit_{CHR}.sh", "w") as f: f.write(template_updated)